<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [1]:
#@title Config and Requirements
!pip install coolname gdown papermill json5
from coolname import generate_slug
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/ipfs"
worker_root = "/content"
print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

     |████████████████████████████████| 143kB 22.7MB/s 
     |████████████████████████████████| 727kB 23.0MB/s 
     |████████████████████████████████| 747kB 29.7MB/s 
     |████████████████████████████████| 122kB 61.9MB/s 
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


---Settings---
IPFS_ROOT: /ipfs
IPFS_PEER: ipfs.pollinations.ai


In [2]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $worker_root
!echo "Created IPFS resource" > $ipfs_root/output/log



rm: cannot remove '/ipfs': No such file or directory


In [3]:
#@title Install IPFS

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm run install_backend
%cd -



/content
Cloning into 'pollinations'...
remote: Enumerating objects: 2286, done.
remote: Counting objects: 100% (678/678), done.
remote: Compressing objects: 100% (543/543), done.
remote: Total 2286 (delta 463), reused 261 (delta 131), pack-reused 1608
Receiving objects: 100% (2286/2286), 19.22 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (1506/1506), done.
/content/pollinations/app
Already up to date.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
Already up to date.

> pollinations@0.4.5 install_backend /content/pollinations/app
> cp -v dist/pollinate.js /usr/local/bin/pollinate

'dist/pollinate.js' -> '/usr/local/bin/pollinate'
/content


In [4]:
!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh > $ipfs_root/output/log &
!sleep 5



--2021-06-12 08:51:54--  https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25904633 (25M) [application/gzip]
Saving to: ‘go-ipfs_v0.8.0_linux-amd64.tar.gz’

go-ipfs_v0.8.0_linu 100%[===================>]  24.70M  --.-KB/s    in 0.1s    

2021-06-12 08:51:55 (214 MB/s) - ‘go-ipfs_v0.8.0_linux-amd64.tar.gz’ saved [25904633/25904633]

go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/LICENSE-APACHE
go-ipfs/LICENSE-MIT
go-ipfs/README.md
Moved ./ipfs to /usr/local/bin
/content
generating ED25519 keypair...done
peer identity: 12D3KooWRFATTCwsNDi5BwSRVVNdKeo8VbadctVRaj5hQBp6pK3j
initializing IPFS node at /root/.ipfs
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15z4bG2X8D2GhfbSXc/readme

nohup: redirecting stderr to stdout


In [5]:
from os.path import join


notebook_path = join(ipfs_root,"notebook.ipynb")



In [6]:
node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

---Node ID---: 12D3KooWRFATTCwsNDi5BwSRVVNdKeo8VbadctVRaj5hQBp6pK3j


In [7]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))

!sleep 5


<IPython.core.display.Javascript object>

In [8]:
# !DEBUG=* pollinate -p {ipfs_root} --ipns --send --once

In [9]:
!DEBUG=* pollinate -p {ipfs_root} --ipns --receive --once

  ipfsConnector NodeID Promise { <pending> } +0ms
  ipfsWatch CLI options {
  path: '/ipfs',
  receive: true,
  send: false,
  once: true,
  ipns: true,
  nodeid: null
} +0ms
  ipfsWatch IPNS activated. subscring to CIDs +0ms
  ipfsConnector Localhost:5001 is reachable. Connecting... +6ms
  ipfsConnector Subscribing to pubsub events from 12D3KooWRFATTCwsNDi5BwSRVVNdKeo8VbadctVRaj5hQBp6pK3j +46ms
  ipfsWatch remoteCID from pubsub QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +19s
  ipfsWatch Processing remote CID QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +1ms
  ipfsState Getting state for CID QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +0ms
  ipfsState:_getIPFSState(/) Getting state for dir root QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +0ms
  utils:ipfsls --- Calling  ipfsls with input QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +0ms
  utils:ipfsls --- In: QmNPhpqy5dLCZMWRNhJDWLwNaDDbZ5pE8p3AW2urEncWAN +0ms
  utils:ipfsls --- Out: ipfsls : [
  {
    name: 'input',
    

In [10]:
from glob import glob
from os.path import basename
from json5 import loads
parameters={}
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f: 
    value = f.read()
    parameters[key] = loads(value)

print("New parameters", parameters)

New parameters {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'text_input': 'Vibrant painting of a synthesizer picnic on a sunny day in the garden of the gasworks with two large chimneys in the background. '}


In [11]:
!nohup pollinate -p {ipfs_root} --ipns --send &

nohup: appending output to 'nohup.out'


In [12]:
#!pip install --upgrade pip

#!pip install --upgrade ipython
#import papermill as pm

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

output_path = ipfs_root+"/output"
#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  cmd = ["papermill", notebook_path, "/content/notebook_output.ipynb"] + params
  print("Running cmd"," ".join(cmd))
  p = Popen(
          cmd, 
          stdout = PIPE, 
          stderr = STDOUT, 
          shell = False, 
          cwd = worker_root
          )


  while True:
    line = p.stdout.readline()#
    if not line: break
    line_text = line.decode("utf-8")

    stdout.write(line)
    with open(f"{ipfs_root}/output/log","a") as log_out:
      if not line_text.startswith("unhandled"):
        log_out.write(line_text)
        log_out.flush()
      
   
        
while True:
  run(parameters)
  !sleep 100

Running {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'text_input': 'Vibrant painting of a synthesizer picnic on a sunny day in the garden of the gasworks with two large chimneys in the background. '}
Running cmd papermill /ipfs/notebook.ipynb /content/notebook_output.ipynb -p text_not disconnected, confusing, incoherent, cluttered, watermarks, text, writing -p text_input Vibrant painting of a synthesizer picnic on a sunny day in the garden of the gasworks with two large chimneys in the background.  -p output_path /ipfs/output
Input Notebook:  /ipfs/notebook.ipynb
Output Notebook: /content/notebook_output.ipynb
Generating grammar tables from /usr/local/lib/python3.7/dist-packages/blib2to3/Grammar.txt
Writing grammar tables to /root/.cache/black/21.6b0/Grammar3.7.10.final.0.pickle
Writing failed: [Errno 2] No such file or directory: '/root/.cache/black/21.6b0/tmpq7edh285'
Generating grammar tables from /usr/local/lib/python3.7/dist-packages/bl

KeyboardInterrupt: ignored